# Lets just do supervised learning

Since we are looking at pairs with random permuations (from dropout), we can't use CCS. This is because our probabilities do not add to one.

People question if unsupervised learning bings anything to the table anyway, so lets start with supervised...


This one is `dual` in that itpasses both parts of the pair into the model at once.

```
x = torch.concat([x0, x1], 1)
y_pred =model(x)
loss(y_pred, y)
```
as opposed to
```
logit0 = model(x0)
logit1 = model(x1)
y_pred = torch.concat([logit0, logit1])
loss(y_pred, y)
```

TODO:
- [ ] fix training curves?
- [ ] fix acc metrics, maybe look at nicks custom metrics


links:
- [loading](https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/alpaca.py)
- [dict](https://github.com/deep-diver/LLM-As-Chatbot/blob/c79e855a492a968b54bac223e66dc9db448d6eba/model_cards.json#L143)
- [prompt_format](https://github.com/deep-diver/PingPong/blob/main/src/pingpong/alpaca.py)

In [1]:

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

from pathlib import Path

import transformers


import lightning.pytorch as pl
# from dataclasses import dataclass

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.preprocessing import RobustScaler

from tqdm.auto import tqdm
import os

from loguru import logger
logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")

transformers.__version__

'4.30.1'

In [2]:
from datasets import load_from_disk, concatenate_datasets
fs = [
    # "./.ds/HuggingFaceH4starchat_beta-None-N_4000-ns_3-mc_0.2-8bf3e5",
    # './.ds/HuggingFaceH4starchat_beta-None-N_8000-ns_3-mc_0.2-2ffc1e',
    # './.ds/HuggingFaceH4starchat_beta-None-N_600-ns_3-mc_0.2-f0d838',
    
    './.ds/HuggingFaceH4starchat_beta-None-N_4000-ns_3-mc_0.2-0392de',
]

# './.ds/HuggingFaceH4starchat_beta-None-N_8000-ns_3-mc_0.2-2ffc1e'
ds = concatenate_datasets([load_from_disk(f) for f in fs])
ds

Dataset({
    features: ['hs1', 'ans1', 'hs2', 'ans2', 'true', 'index', 'version', 'info', 'input_truncated', 'prob_y', 'prob_n', 'text_ans', 'input_text'],
    num_rows: 8000
})

In [3]:
# fs

In [4]:
# from datasets import load_from_disk, Dataset, load_dataset, load_dataset_builder
# f="./.ds/HuggingFaceH4starchat_beta-None-N_4000-ns_3-mc_0.2-8bf3e5_builder/"
# # ds = Dataset.from_file(f)
# fs=[str(s) for s in Path(f).glob('*.arrow')]
# ds = load_dataset(f, data_files=fs, split="train")
# # load_dataset_builder(f)

## Lightning DataModule

What are we detecting?

We have a pair of inputs, for differen't dropouts. During training we know that one is in the direciton of truth/deception/error

During inferance we also have a pair but don't know which is slower to what we want.

In [5]:
def ds_info2df(ds):
    d = pd.DataFrame(list(ds['info']))
    return d

def ds2df(ds):
    df = ds_info2df(ds)
    df_ans = ds.select_columns(['ans1', 'ans2', 'true']).with_format("numpy").to_pandas()
    df = pd.concat([df, df_ans], axis=1)
    
    # derived
    df['dir_true'] = df['ans2'] - df['ans1']
    df['conf'] = (df['ans1']-df['ans2']).abs()  
    df['llm_prob'] = (df['ans1']+df['ans2'])/2
    df['llm_ans'] = df['llm_prob']>0.5
    return df

class imdbHSDataModule(pl.LightningDataModule):

    def __init__(self,
                 ds,
                 batch_size=32,
                ):
        super().__init__()
        self.save_hyperparameters(ignore=["ds"])
        self.ds = ds

    def setup(self, stage: str):
        h = self.hparams
        
        # extract data set into N-Dim tensors and 1-d dataframe
        self.ds_hs = (
            self.ds.select_columns(['hs1', 'hs2'])
            .with_format("numpy")
        )
        self.df = ds2df(ds)
        
        self.y = self.df['true_answer'].astype(np.float32).values # detection of true answer
        self.y = (self.df['true_answer'] == (self.df['dir_true']>0)).values # is the direction in the dir of truth
        self.y = (self.df['lie'] * ((self.df['llm_ans']>0.5)==self.df['desired_answer']) *  (self.df['dir_true']>0)).values # deception
        self.df['y'] = y
        
        b = len(self.ds_hs)
        self.hs1 = self.ds_hs['hs1'].reshape((b, -1))#.numpy()
        self.hs2 = self.ds_hs['hs2'].reshape((b, -1))#.numpy()        
        self.ans1 = self.df['ans1'].values
        self.ans2 = self.df['ans2'].values

        # let's create a simple 50/50 train split (the data is already randomized)
        n = len(self.y)
        self.val_split = vs = int(n * 0.5)
        self.test_split = ts = int(n * 0.75)
        hs1_train, hs2_train, y_train = self.hs1[:vs], self.hs2[:vs], self.y[:vs]
        hs1_val, hs2_val, y_val = self.hs1[vs:ts], self.hs2[vs:ts], self.y[vs:ts]
        hs1_test, hs2_test, y_test = self.hs1[ts:],self. hs2[ts:], self.y[ts:]

        self.ds_train = TensorDataset(torch.from_numpy(hs1_train).float(),
                                      torch.from_numpy(hs2_train).float(),
                                      torch.from_numpy(y_train).float())

        self.ds_val = TensorDataset(torch.from_numpy(hs1_val).float(),
                                    torch.from_numpy(hs2_val).float(),
                                    torch.from_numpy(y_val).float())

        self.ds_test = TensorDataset(torch.from_numpy(hs1_test).float(),
                                     torch.from_numpy(hs2_test).float(),
                                     torch.from_numpy(y_test).float())

    def train_dataloader(self):
        return DataLoader(self.ds_train,
                          batch_size=self.hparams.batch_size,
                          shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.ds_val, batch_size=self.hparams.batch_size)

    def test_dataloader(self):
        return DataLoader(self.ds_test, batch_size=self.hparams.batch_size)


In [6]:
batch_size = 128
# test and cache
dm = imdbHSDataModule(ds, batch_size=batch_size)
dm.setup('train')

dl_val = dm.val_dataloader()
dl_train = dm.train_dataloader()
b = next(iter(dl_train))
b

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 batch_size = 128                                                                            │
│    2 # test and cache                                                                            │
│    3 dm = imdbHSDataModule(ds, batch_size=batch_size)                                            │
│ ❱  4 dm.setup('train')                                                                           │
│    5                                                                                             │
│    6 dl_val = dm.val_dataloader()                                                                │
│    7 dl_train = dm.train_dataloader()                                                            │
│                                                                                                  │
│ in setup:40                                                                                      │
│                                                                                                  │
│   37 │   │   self.y = self.df['true_answer'].astype(np.float32).values # detection of true an    │
│   38 │   │   self.y = (self.df['true_answer'] == (self.df['dir_true']>0)).values # is the dir    │
│   39 │   │   self.y = (self.df['lie'] * ((self.df['llm_ans']>0.5)==self.df['desired_answer'])    │
│ ❱ 40 │   │   self.df['y'] = y                                                                    │
│   41 │   │                                                                                       │
│   42 │   │   b = len(self.ds_hs)                                                                 │
│   43 │   │   self.hs1 = self.ds_hs['hs1'].reshape((b, -1))#.numpy()                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'y' is not defined

In [7]:

hss1 = dm.hs1
hss2 = dm.hs2
ans_1 = dm.ans1
ans_2 = dm.ans2
y = dm.y
print('y_balance', y.mean())
df = dm.df
df

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 hss1 = dm.hs1                                                                                │
│   2 hss2 = dm.hs2                                                                                │
│   3 ans_1 = dm.ans1                                                                              │
│   4 ans_2 = dm.ans2                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'imdbHSDataModule' object has no attribute 'hs1'

# Data prep

We do two inferences on the same inputs. Since we have dropout enabled, even during inference, we get two slightly different hidden states `hs1` and `hs2`, and two slightly different probabilities for our yes and no output tokens `p1` `p2`. We also have the true answer `t`

So there are a few ways we can set up the problem. 

We can vary x:
- `model(hs1)-model(hs2)=y`
- `model(hs1-hs2)==y`

And we can try differen't y's:
- direction with a ranked loss. This could be unsupervised.
- magnitude with a regression loss
- vector (direction and magnitude) with a regression loss

# QC: Linear supervised probes


Let's verify that the model's representations are good

Before trying CCS, let's make sure there exists a direction that classifies examples as true vs false with high accuracy; if supervised logistic regression accuracy is bad, there's no hope of unsupervised CCS doing well.

Note that because logistic regression is supervised we expect it to do better but to have worse generalisation that equivilent unsupervised methods. However in this case CSS is using a deeper model so it is more complicated.


## Try a classification of direction to truth

In [8]:

n = len(df)

# Define X and y
X = hss1-hss2

# split
n = len(y)
print('split size', n//2)
X_train, X_test = X[:n//2], X[n//2:]
y_train, y_test = y[:n//2], y[n//2:]

# scale
scaler = RobustScaler()
scaler.fit(X_train)
X_train2 = scaler.transform(X_train)
X_test2 = scaler.transform(X_test)

lr = LogisticRegression(class_weight="balanced", penalty="l2", max_iter=380)
lr.fit(X_train2, y_train>0)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 n = len(df)                                                                                 │
│    2                                                                                             │
│    3 # Define X and y                                                                            │
│    4 X = hss1-hss2                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df' is not defined

In [9]:
print("Logistic cls acc: {:2.2%} [TRAIN]".format(lr.score(X_train2, y_train>0)))
print("Logistic cls acc: {:2.2%} [TEST]".format(lr.score(X_test2, y_test>0)))

m = df['lie'][n//2:]
y_test_pred = lr.predict(X_test2)
acc_w_lie = ((y_test_pred[m]>0)==(y_test[m]>0)).mean()
acc_wo_lie = ((y_test_pred[~m]>0)==(y_test[~m]>0)).mean()
print(f'test acc w lie {acc_w_lie:2.2%}')
print(f'test acc wo lie {acc_wo_lie:2.2%}')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 print("Logistic cls acc: {:2.2%} [TRAIN]".format(lr.score(X_train2, y_train>0)))            │
│    2 print("Logistic cls acc: {:2.2%} [TEST]".format(lr.score(X_test2, y_test>0)))               │
│    3                                                                                             │
│    4 m = df['lie'][n//2:]                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'lr' is not defined

In [10]:
df_info_test = df.iloc[n//2:].copy()
y_pred = lr.predict(X_test2)
df_info_test['inner_truth'] = y_pred
df_info_test

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df_info_test = df.iloc[n//2:].copy()                                                         │
│   2 y_pred = lr.predict(X_test2)                                                                 │
│   3 df_info_test['inner_truth'] = y_pred                                                         │
│   4 df_info_test                                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df' is not defined

### Result, detecting deception?

In [11]:
lie_pred = df_info_test['inner_truth']==df_info_test['true_answer']
lie_true = df_info_test['lie']
acc_lie = accuracy_score(lie_pred, lie_true)
print(f"model can detect lies with acc {acc_lie:2.2%}")
print(f"w lies {sum(lie_true)}/{len(lie_true)} test rows")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 lie_pred = df_info_test['inner_truth']==df_info_test['true_answer']                          │
│   2 lie_true = df_info_test['lie']                                                               │
│   3 acc_lie = accuracy_score(lie_pred, lie_true)                                                 │
│   4 print(f"model can detect lies with acc {acc_lie:2.2%}")                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_info_test' is not defined

# LightningModel

In [12]:
class MLPProbe(nn.Module):
    def __init__(self, c_in, depth=0, hs=16, dropout=0):
        super().__init__()

        layers = [
            nn.Dropout1d(dropout),
            nn.BatchNorm1d(c_in),  # this will normalise the inputs
            nn.Linear(c_in, hs),
            nn.Dropout1d(dropout),
        ]
        for _ in range(depth):
            layers += [
                nn.Linear(hs, hs),
                nn.ReLU(),
                nn.Dropout1d(dropout),
            ]
        layers += [nn.Linear(hs, 2)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


In [13]:
class_weights = 1/torch.Tensor(pd.Series(y).value_counts(True).values)
class_weights /= class_weights.sum()
class_weights

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 class_weights = 1/torch.Tensor(pd.Series(y).value_counts(True).values)                       │
│   2 class_weights /= class_weights.sum()                                                         │
│   3 class_weights                                                                                │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'y' is not defined

In [14]:
from pytorch_optimizer import Ranger21
import torchmetrics
# from focal_loss.focal_loss import FocalLoss

from torchmetrics import Metric, MetricCollection, Accuracy, AUROC
    
class CSS(pl.LightningModule):
    def __init__(self, c_in, total_steps, depth=1, hs=16, lr=4e-3, weight_decay=1e-9, dropout=0):
        super().__init__()
        self.probe = MLPProbe(c_in*2, depth=depth, dropout=dropout, hs=hs)
        self.save_hyperparameters()
        
        # self.loss_fn = FocalLoss(0.7)
        self.loss_fn = nn.CrossEntropyLoss(class_weights)
        
        # metrics for each stage
        metrics_template = MetricCollection({
            'acc': Accuracy(task="multiclass", num_classes=2), 
            'auroc': AUROC(task="multiclass", num_classes=2)
        })
        self.metrics = torch.nn.ModuleDict({
            f'metrics_{stage}': metrics_template.clone(prefix=stage+'/') for stage in ['train', 'val', 'test']
        })
        
    def forward(self, x):
        return self.probe(x)
        
    def _step(self, batch, batch_idx, stage='train'):
        x0, x1, y = batch
        x = torch.concatenate([x0, x1], 1)
        logits = self(x)
        y_pred = F.softmax(logits, -1)
        if stage=='pred':
            return y_pred
        
        loss = self.loss_fn(y_pred, y.long())
        self.log(f"{stage}/loss", loss)
        
        m = self.metrics[f'metrics_{stage}']
        m(y_pred, y.long())
        self.log_dict(m, on_epoch=True, on_step=False)
        return loss
    
    def training_step(self, batch, batch_idx=0, dataloader_idx=0):
        return self._step(batch, batch_idx)
    
    def validation_step(self, batch, batch_idx=0):
        return self._step(batch, batch_idx, stage='val')
    
    def predict_step(self, batch, batch_idx=0, dataloader_idx=0):
        return self._step(batch, batch_idx, stage='pred').cpu().detach()
    
    def test_step(self, batch, batch_idx=0, dataloader_idx=0):
        return self._step(batch, batch_idx, stage='test')
    
    def configure_optimizers(self):
        """use ranger21 from  https://github.com/kozistr/pytorch_optimizer"""
        optimizer = Ranger21(
            self.parameters(),
            lr=self.hparams.lr,
            weight_decay=self.hparams.weight_decay,       
            num_iterations=self.hparams.total_steps,
        )
        return optimizer
    
    

# Run

In [15]:
# quiet please
torch.set_float32_matmul_precision('medium')

import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*F-score.*")

## Prep dataloader/set

In [16]:
# # split
# X = hss1-hss2
# y = (df['true_answer'] == (df['dir_true']>0)).values # does this dropout take it in the direction of truth
# y = df['lie'] * ((df['llm_ans']>0.5)==df['desired_answer']) # deception
# n = len(y)
# print('split size', n//2)

# neg_hs_train = hss1[:n//2]
# pos_hs_train = hss2[:n//2]

# neg_hs_val = hss1[n//2:]
# pos_hs_val = hss2[n//2:]

# y_train, y_val = y[:n//2], y[n//2:]

In [17]:
dl_train = dm.train_dataloader()
dl_val = dm.val_dataloader()
b = next(iter(dl_train))
# b

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dl_train = dm.train_dataloader()                                                             │
│   2 dl_val = dm.val_dataloader()                                                                 │
│   3 b = next(iter(dl_train))                                                                     │
│   4 # b                                                                                          │
│                                                                                                  │
│ in train_dataloader:69                                                                           │
│                                                                                                  │
│   66 │   │   │   │   │   │   │   │   │    torch.from_numpy(y_test).float())                      │
│   67 │                                                                                           │
│   68 │   def train_dataloader(self):                                                             │
│ ❱ 69 │   │   return DataLoader(self.ds_train,                                                    │
│   70 │   │   │   │   │   │     batch_size=self.hparams.batch_size,                               │
│   71 │   │   │   │   │   │     shuffle=True)                                                     │
│   72                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'imdbHSDataModule' object has no attribute 'ds_train'

In [18]:
# init the model
max_epochs = 16
c_in = b[0].shape[-1]
print(b[0].shape)
net = CSS(c_in=c_in, total_steps=max_epochs*len(dl_train), depth=1, hs=8, lr=1e-3, weight_decay=1e-4, dropout=0.1)
net

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 # init the model                                                                             │
│   2 max_epochs = 16                                                                              │
│ ❱ 3 c_in = b[0].shape[-1]                                                                        │
│   4 print(b[0].shape)                                                                            │
│   5 net = CSS(c_in=c_in, total_steps=max_epochs*len(dl_train), depth=1, hs=8, lr=1e-3, weigh     │
│   6 net                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'b' is not defined

In [19]:
# # DEBUG
# with torch.no_grad():
#     b = next(iter(dl_train))
#     b2 = [bb.to(net.device) for bb in b]
#     x = torch.concatenate([b2[0], b2[1]], 1)
#     y = net(x)
# y

In [20]:
# # DEBUG
# trainer = pl.Trainer(fast_dev_run=2)
# trainer.fit(model=net, train_dataloaders=dl_train)

In [21]:
trainer = pl.Trainer(precision="bf16",
                     max_epochs=max_epochs, log_every_n_steps=5)
trainer.fit(model=net, train_dataloaders=dl_train, val_dataloaders=dl_val)

/home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/lightning/fabric/connector.py:562: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by de

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 trainer = pl.Trainer(precision="bf16",                                                       │
│   2 │   │   │   │   │    max_epochs=max_epochs, log_every_n_steps=5)                             │
│ ❱ 3 trainer.fit(model=net, train_dataloaders=dl_train, val_dataloaders=dl_val)                   │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'net' is not defined

# Read hist

In [22]:
# import pytorch_lightning as pl
from lightning.pytorch.loggers.csv_logs import CSVLogger
from pathlib import Path
import pandas as pd

def read_metrics_csv(metrics_file_path):
    df_hist = pd.read_csv(metrics_file_path)
    df_hist["epoch"] = df_hist["epoch"].ffill()
    df_histe = df_hist.set_index("epoch").groupby("epoch").mean()
    return df_histe
        
df_hist = read_metrics_csv(trainer.logger.experiment.metrics_file_path).ffill().bfill()
df_hist


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   df_histe = df_hist.set_index("epoch").groupby("epoch").mean()                           │
│   10 │   return df_histe                                                                         │
│   11                                                                                             │
│ ❱ 12 df_hist = read_metrics_csv(trainer.logger.experiment.metrics_file_path).ffill().bfill()     │
│   13 df_hist                                                                                     │
│   14                                                                                             │
│                                                                                                  │
│ in read_metrics_csv:7                                                                            │
│                                                                                                  │
│    4 import pandas as pd                                                                         │
│    5                                                                                             │
│    6 def read_metrics_csv(metrics_file_path):                                                    │
│ ❱  7 │   df_hist = pd.read_csv(metrics_file_path)                                                │
│    8 │   df_hist["epoch"] = df_hist["epoch"].ffill()                                             │
│    9 │   df_histe = df_hist.set_index("epoch").groupby("epoch").mean()                           │
│   10 │   return df_histe                                                                         │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/io/parsers/readers.py:912   │
│ in read_csv                                                                                      │
│                                                                                                  │
│    909 │   )                                                                                     │
│    910 │   kwds.update(kwds_defaults)                                                            │
│    911 │                                                                                         │
│ ❱  912 │   return _read(filepath_or_buffer, kwds)                                                │
│    913                                                                                           │
│    914                                                                                           │
│    915 # iterator=True -> TextFileReader                                                         │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/io/parsers/readers.py:577   │
│ in _read                                                                                         │
│                                                                                                  │
│    574 │   _validate_names(kwds.get("names", None))                                              │
│    575 │                                                                                         │
│    576 │   # Create the parser.                                                                  │
│ ❱  577 │   parser = TextFileReader(filepath_or_buffer, **kwds)                                   │
│    578 │                                                                                         │
│    579 │   if chunksize or iterator:                       

In [23]:
for key in ['loss', 'acc', 'auroc']:
    df_hist[[c for c in df_hist.columns if key in c]].plot()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 for key in ['loss', 'acc', 'auroc']:                                                         │
│ ❱ 2 │   df_hist[[c for c in df_hist.columns if key in c]].plot()                                 │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_hist' is not defined

## Predict

In [24]:
dl_test = dm.test_dataloader()
rs = trainer.test(net, dataloaders=[dl_train, dl_val, dl_test])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dl_test = dm.test_dataloader()                                                               │
│   2 rs = trainer.test(net, dataloaders=[dl_train, dl_val, dl_test])                              │
│   3                                                                                              │
│                                                                                                  │
│ in test_dataloader:77                                                                            │
│                                                                                                  │
│   74 │   │   return DataLoader(self.ds_val, batch_size=self.hparams.batch_size)                  │
│   75 │                                                                                           │
│   76 │   def test_dataloader(self):                                                              │
│ ❱ 77 │   │   return DataLoader(self.ds_test, batch_size=self.hparams.batch_size)                 │
│   78                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'imdbHSDataModule' object has no attribute 'ds_test'

In [25]:
dl_test = dm.test_dataloader()
r = trainer.predict(net, dataloaders=dl_test)
y_test_pred = np.concatenate(r)
y_test_pred.shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dl_test = dm.test_dataloader()                                                               │
│   2 r = trainer.predict(net, dataloaders=dl_test)                                                │
│   3 y_test_pred = np.concatenate(r)                                                              │
│   4 y_test_pred.shape                                                                            │
│                                                                                                  │
│ in test_dataloader:77                                                                            │
│                                                                                                  │
│   74 │   │   return DataLoader(self.ds_val, batch_size=self.hparams.batch_size)                  │
│   75 │                                                                                           │
│   76 │   def test_dataloader(self):                                                              │
│ ❱ 77 │   │   return DataLoader(self.ds_test, batch_size=self.hparams.batch_size)                 │
│   78                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'imdbHSDataModule' object has no attribute 'ds_test'

In [26]:
df_test = dm.df.iloc[dm.test_split:].copy()
df_test['probe_pred'] = y_test_pred.argmax(-1)
df_test['probe_prob'] = y_test_pred[:, 1]
df_test['llm_prob'] = (df_test['ans1']+df_test['ans2'])/2
df_test['llm_ans'] = df_test['llm_prob']>0.5
df_test['conf'] = (df_test['ans1']-df_test['ans2']).abs()

y_true = dl_test.dataset.tensors[2].numpy()
assert ((df_test['y'].values>0.5)==(y_true>0.5)).all(), 'check it all lines up'

df_test

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 df_test = dm.df.iloc[dm.test_split:].copy()                                                 │
│    2 df_test['probe_pred'] = y_test_pred.argmax(-1)                                              │
│    3 df_test['probe_prob'] = y_test_pred[:, 1]                                                   │
│    4 df_test['llm_prob'] = (df_test['ans1']+df_test['ans2'])/2                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'imdbHSDataModule' object has no attribute 'test_split'

In [27]:
def get_acc_subset(df, query):
    df_s = df.query(query)
    acc = (df_s['probe_pred']==df_s['y']).mean()
    print(f"acc={acc:2.2%} [{query}]")
    
print('probe results on subsets of the data')
get_acc_subset(df_test, 'lie==True') # it was ph told to lie
get_acc_subset(df_test, 'lie==False') # it was told not to lie
get_acc_subset(df_test, 'llm_ans==true_answer') # the llm gave the true ans
get_acc_subset(df_test, 'llm_ans==desired_answer') # the llm gave the desired ans
get_acc_subset(df_test, 'lie==True & llm_ans==desired_answer') # it was told to lie, and it did lie
get_acc_subset(df_test, 'lie==True & llm_ans!=desired_answer')

probe results on subsets of the data


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4 │   print(f"acc={acc:2.2%} [{query}]")                                                      │
│    5                                                                                             │
│    6 print('probe results on subsets of the data')                                               │
│ ❱  7 get_acc_subset(df_test, 'lie==True') # it was ph told to lie                                │
│    8 get_acc_subset(df_test, 'lie==False') # it was told not to lie                              │
│    9 get_acc_subset(df_test, 'llm_ans==true_answer') # the llm gave the true ans                 │
│   10 get_acc_subset(df_test, 'llm_ans==desired_answer') # the llm gave the desired ans           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_test' is not defined

In [28]:
acc_truth = (df_test['probe_pred']==(df_test['llm_ans']>0.5)).mean()
print(f"lightning model acc at predicting the models public answer (may not what it's trained for): {acc_truth:2.2%}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 acc_truth = (df_test['probe_pred']==(df_test['llm_ans']>0.5)).mean()                         │
│   2 print(f"lightning model acc at predicting the models public answer (may not what it's tr     │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_test' is not defined

## Task results

E.g. how well does the underlying language model do on the task

In [29]:
print('Can the model lie?')
c_in = df_test.query('lie==True')
(c_in['desired_answer']==c_in['llm_ans']).mean()

Can the model lie?


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 print('Can the model lie?')                                                                  │
│ ❱ 2 c_in = df_test.query('lie==True')                                                            │
│   3 (c_in['desired_answer']==c_in['llm_ans']).mean()                                             │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_test' is not defined